In [1]:
import requests
from bs4 import BeautifulSoup
import re
from typing import List
import pandas as pd

__all_posts: List[dict] = []
KEYWORDS = ['python', 'парсинг']

def read_data_from_remote_resource(url):
    return BeautifulSoup(requests.get(url).text, 'html.parser')

def load_available_pages(habr_all_page):
    def find_pagination_elements(html_page):
        return list(map(lambda x: x.find_all('a'), html_page.find_all('ul', id='nav-pagess')))[0]
    def map_pagination_elements_to_pages(elements_list):
        return list(map(lambda element: element.attrs.get('href'), elements_list))
    return map_pagination_elements_to_pages(find_pagination_elements(habr_all_page))

def find_last_page_num(available_pages_list):
    return int(list(map(lambda x: re.findall('\d+', x), available_pages_list))[len(available_pages_list) - 1][0])

def extract_posts_from_page_content(html_page):
    def extract_posts_content_block(habr_all_posts_page):
        return habr_all_posts_page.find_all('div', class_='posts_list')
    def extract_only_posts(habr_posts_block):
        return list(map(lambda element: element.find_all('article', class_='post_preview'), habr_posts_block))[0]
    return extract_only_posts(extract_posts_content_block(html_page))

def get_post_content(post: BeautifulSoup):
    def get_title(post_data: BeautifulSoup):
        return post_data.find_all('a', class_='post__title_link')[0].text
    def get_content(post_data: BeautifulSoup):
        return post_data.find_all('div', class_='post__text')[0].text
    def get_link(post_data: BeautifulSoup):
        return post_data.find_all('a', class_='post__title_link')[0].attrs.get('href')
    return {
        'title': get_title(post),
        'content': get_content(post),
        'link': get_link(post)
    }

def _is_contains_keywods(post_dict: dict):
    for keyword in KEYWORDS:
        if keyword.lower() in str(post_dict.get('title')).lower() \
                or keyword.lower() in str(post_dict.get('content')).lower():
            return True
    return False

def add_posts_from_collection(posts_collection: list):
    for post in posts_collection:
        _add_post(post)

def _add_post(post: BeautifulSoup):
    post_dict = get_post_content(post)
    if _is_contains_keywods(post_dict):
        __all_posts.append(post_dict)

first_page = read_data_from_remote_resource('https://habr.com/ru/all/')
max_page_number = find_last_page_num(load_available_pages(first_page))

add_posts_from_collection(extract_posts_from_page_content(first_page))

for i in range(2, max_page_number + 1):
    page = read_data_from_remote_resource('https://habr.com/ru/all/page' + str(i) + '/')
    add_posts_from_collection(extract_posts_from_page_content(page))

pd.DataFrame(__all_posts)

,title,content,link
0,Бесплатный удобный ETL инструмент с открытым к...,\nСегодня я хочу рассказать об open-source инс...,https://habr.com/ru/company/visiology/blog/536...
1,Micropyserver. Реализуем Basic Auth для IoT ус...,\nЧасто нужно закрыть доступ к веб-интерфейсу ...,https://habr.com/ru/post/544240/
2,Тестирование скриншотами,\nЗдравствуйте! Сегодня хочу рассказать о наше...,https://habr.com/ru/company/otus/blog/545084/
3,Стриминг Edge2AI на NVIDIA JETSON Nano 2Гб с и...,\nСтриминг Edge2AI на новой карте NVIDIA JETSO...,https://habr.com/ru/company/cloudera/blog/543116/
4,Как за 60$ создать систему распознавания лиц с...,\nНовый набор инструментов для разработчиков N...,https://habr.com/ru/company/skillfactory/blog/...
5,Книга «Глубокое обучение: легкая разработка пр...,"\n Привет, Хаброжители! Взрывной интерес к ней...",https://habr.com/ru/company/piter/blog/544822/
6,Популярность BPM в разных жанрах музыки. Pytho...,"\nНесколько лет назад, занимался изучением тео...",https://habr.com/ru/post/544540/
7,Архитектура в Django проектах — как выжить,"\nДумаю, ни для кого не секрет, что в разговор...",https://habr.com/ru/company/vivid_money/blog/5...
8,Облегчаем себе жизнь с помощью BeautifulSoup4,\n Приветствую всех. В этой статье мы сде...,https://habr.com/ru/post/544828/
9,Как улучшить резюме с помощью алгоритмов обраб...,\nРекрутеры используют всё более сложное ПО и ...,https://habr.com/ru/company/skillfactory/blog/...


In [ ]:
import pandas as pd
import time
import pandas as pd
import requests
import json

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
emails = ['barkal97@mail.ru','barkal4ik@gmail.com']

for email in emails:
    params = {'emailAddresses': [email]}
    res = requests.post(URL, data = json.dumps(params), headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-avast'})
    time.sleep(0.3)
    res.json()
    table = pd.DataFrame(res.json()['breaches'])
    transpon_table =  table.transponse()
    table_1 = transpon_table.loc[:,['publishDate', 'site', 'description']].head()
table_1